In [1]:
#import package
from cobra import Model, Reaction, Metabolite
import cobra
import pandas as pd
import numpy as np
from pandas import ExcelWriter
from pandas import ExcelFile

from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)
from cobra.util.solver import linear_reaction_coefficients
import numpy as np
from sklearn import datasets


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt


In [ ]:
# path to sheet: C:\Users\zafri\Desktop\BENG 213\BENG 213- Pangenome analysis.xlsx

In [3]:
base_matrix = pd.read_excel('BENG 213- Pangenome analysis.xlsx', sheetname='Matrix_w_Genes',index_col=0)

In [4]:
base_matrix.columns

Index([u'hypothetical protein', u'hypothetical protein.1', u'Toxin B',
       u'hypothetical protein.2', u'Hemin receptor', u'hypothetical protein.3',
       u'Ribonuclease HI', u'hypothetical protein.4',
       u'Ribosomal RNA large subunit methyltransferase H',
       u'Endoribonuclease YbeY',
       ...
       u'hypothetical protein.9586', u'Nif-specific regulatory protein',
       u'putative RNA polymerase sigma factor FecI.1',
       u'hypothetical protein.9587', u'hypothetical protein.9588',
       u'hypothetical protein.9589', u'hypothetical protein.9590',
       u'Antirestriction protein KlcA.11', u'hypothetical protein.9591',
       u'AIEC'],
      dtype='object', length=16089)

In [6]:
Num_hyp_prot=0
Non_hyp_prot_list=[]
hyp_prot_list=[]
all_col=[]
for columns in base_matrix.columns:
    all_col.append(columns)
    if columns.find("hypothetical protein")==-1:
        Non_hyp_prot_list.append(columns)
    else:
        Num_hyp_prot=Num_hyp_prot+1
        hyp_prot_list.append(columns)
print Num_hyp_prot

9592


In [7]:
strain_list=[]
for i in base_matrix.index:
    strain_list.append(i)
print len(strain_list)
print strain_list

108
[u'12-1-TI12', u'12-2-TI13', u'18-3-TI5', u'18-4-TI12', u'33-1-TI5', u'36-1-TI13', u'39-2-TI18', u'3_2_53FAA', u'41-2-TI13', u'46-1-TI2', u'52-1-TI13', u'52-2-TI10', u'54-1-TI6', u'55-1-AU4', u'55-1-TI19', u'57-3-TI5', u'60-1-TI1', u'61-1-TI1', u'62-2-TI6', u'63-1-TI1', u'69-1-AU1', u'69-1-TI1', u'70-2-TI12', u'72-6-TI12', u'CD_1IM_3', u'CD_34_LN', u'CD_64_LN', u'E_coli_G58-1', u'E_coli_Nissle_1917', u'Escherichia_coli_042', u'Escherichia_coli_536', u'Escherichia_coli_55989', u'Escherichia_coli_ABU_83972', u'Escherichia_coli_APEC_O1', u'Escherichia_coli_ATCC_8739', u'Escherichia_coli_BL21_AM946981', u'Escherichia_coli_BL21_BL21-GoldpLysS_AG', u'Escherichia_coli_BL21_CP001509', u'Escherichia_coli_BW2952', u'Escherichia_coli_B_str_REL606', u'Escherichia_coli_CFT073', u'Escherichia_coli_DH1', u'Escherichia_coli_DH1_ME8569', u'Escherichia_coli_E24377A', u'Escherichia_coli_ED1a', u'Escherichia_coli_HS', u'Escherichia_coli_IAI1', u'Escherichia_coli_IAI39', u'Escherichia_coli_IHE3034', u'

In [8]:
gene_list=list(base_matrix.columns.values)
print gene_list

[u'hypothetical protein', u'hypothetical protein.1', u'Toxin B', u'hypothetical protein.2', u'Hemin receptor', u'hypothetical protein.3', u'Ribonuclease HI', u'hypothetical protein.4', u'Ribosomal RNA large subunit methyltransferase H', u'Endoribonuclease YbeY', u'Transcriptional regulator MntR', u'Outer membrane lipoprotein SlyB', u'hypothetical protein.5', u'hypothetical protein.6', u'hypothetical protein.7', u'putative 9-O-acetyl-N-acetylneuraminic acid deacetylase', u'2,3-diketo-L-gulonate TRAP transporter small permease protein YiaM', u'Protein-export protein SecB', u'hypothetical protein.8', u'hypothetical protein.9', u'hypothetical protein.10', u'Type 4 prepilin-like proteins leader peptide-processing enzyme', u'Toxin-antitoxin biofilm protein TabA', u'hypothetical protein.11', u'hypothetical protein.12', u'hypothetical protein.13', u'Dihydroxy-acid dehydratase', u'hypothetical protein.14', u'hypothetical protein.15', u'hypothetical protein.16', u'hypothetical protein.17', u'hyp

In [26]:
base_matrix
base=base_matrix.as_matrix()
base.shape

(108L, 16089L)

In [9]:
core_genes=[]
c=0
for i in range(16089):
    if sum(base_matrix.iloc[:,i])==108:
        c=c+1
        core_genes.append(gene_list[i])
print c

2346


In [10]:
print core_genes


[u'Ribonuclease HI', u'Ribosomal RNA large subunit methyltransferase H', u'Endoribonuclease YbeY', u'Transcriptional regulator MntR', u'Outer membrane lipoprotein SlyB', u'Protein-export protein SecB', u'hypothetical protein.8', u'hypothetical protein.25', u'Murein DD-endopeptidase MepS/Murein LD-carboxypeptidase', u'Toxin-antitoxin biofilm protein TabA.1', u'Ascorbate-specific phosphotransferase enzyme IIA component', u'Anaerobic ribonucleoside-triphosphate reductase-activating protein', u'Leucine-responsive regulatory protein', u'hypothetical protein.50', u'Phosphatase NudJ', u'tRNA threonylcarbamoyladenosine biosynthesis protein TsaE', u'Aspartate carbamoyltransferase regulatory chain', u'DNA topoisomerase 3', u'Transcriptional regulator MraZ', u'Xanthine phosphoribosyltransferase', u'Inner membrane protein YbbJ', u'Methylglyoxal synthase', u"Deoxyuridine 5'-triphosphate nucleotidohydrolase", u'Regulatory protein AsnC', u'RNA polymerase-binding transcription factor DksA', u'3-hydrox

In [24]:
from sklearn.decomposition import TruncatedSVD
SVD=[]
for i in range(30):
    svd = TruncatedSVD(n_components=i, n_iter=7, random_state=42)
    svd.fit(base_matrix)
    print "%s: %s: %s" % (i,svd.explained_variance_ratio_.sum(),svd.singular_values_)
    #print "%s: %s: %s: %s" % (i,svd.explained_variance_ratio_,svd.explained_variance_ratio_.sum(),svd.singular_values_)
    #print zip(i,svd.explained_variance_ratio_.sum())
    SVD.extend([[i,svd.explained_variance_ratio_.sum(),svd.singular_values_]])

0: 0.0: []
1: 0.00817597390762: [ 637.11457159]
2: 0.2220918996: [ 637.11457159  131.94278762]
3: 0.281131131418: [ 637.11457159  131.94278762   69.28632204]
4: 0.331933003848: [ 637.11457159  131.94278762   69.28632365   64.26933099]
5: 0.370515575071: [ 637.11457159  131.94278762   69.28632368   64.26932982   56.01536105]
6: 0.408028472403: [ 637.11457159  131.94278762   69.28632369   64.26933109   56.01537072
   55.23086213]
7: 0.438326083484: [ 637.11457159  131.94278762   69.28632369   64.26933112   56.01537095
   55.23086135   49.63238129]
8: 0.467642104621: [ 637.11457159  131.94278762   69.28632369   64.26933118   56.01537054
   55.23086239   49.6323713    48.82184045]
9: 0.494344382742: [ 637.11457159  131.94278762   69.28632369   64.26933113   56.01536929
   55.23086136   49.63236002   48.82181658   46.594627  ]
10: 0.520562680087: [ 637.11457159  131.94278762   69.28632369   64.26933119   56.0153739
   55.23086284   49.6323752    48.82182286   46.59479738   46.17004298]
11: 

In [25]:

label=['i', 'explained_variance_ratio_sum','singular_values']

In [26]:

SVD2=pd.DataFrame.from_records(SVD, columns=label)
SVD2

,i,explained_variance_ratio_sum,singular_values
0,0,0.000000,[]
1,1,0.008176,[637.114571587]
2,2,0.222092,"[637.114571587, 131.942787617]"
3,3,0.281131,"[637.114571587, 131.942787617, 69.2863220394]"
4,4,0.331933,"[637.114571587, 131.942787617, 69.2863236533, ..."
5,5,0.370516,"[637.114571587, 131.942787617, 69.2863236785, ..."
6,6,0.408028,"[637.114571587, 131.942787617, 69.2863236879, ..."
7,7,0.438326,"[637.114571587, 131.942787617, 69.2863236894, ..."
8,8,0.467642,"[637.114571587, 131.942787617, 69.286323687, 6..."
9,9,0.494344,"[637.114571587, 131.942787617, 69.2863236854, ..."


In [ ]:
for i in range(20)
    print i 

In [ ]:
fig, ax = plt.subplots(figsize=(100,100))       

In [ ]:
ax = sns.heatmap(base,cmap="YlGnBu",linewidths=.001,yticklabels=strain_list,xticklabels=all_col)
plt.show()
fig.savefig("output2.png") 

In [ ]:
#reduce dimensions by categories 
reduced_base=base_matrix
strains_not_found=[]
strains_found=[]
for strains in hyp_prot_list:
    if strains in base_matrix.columns:
        reduced_base=reduced_base.drop(strains, axis=1)
        strains_found.append(strains)
    else: 
        strains_not_found.append(strains)
        #print strains
reduced_base
    
        

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))       

In [ ]:
ax = sns.heatmap(reduced_base,cmap="YlGnBu", yticklabels=strain_list,xticklabels=strains_found)
plt.show()
fig.savefig("output2.png") 

In [ ]:
g = sns.clustermap(reduced_base)
plt.show()


In [ ]:
META = pd.read_excel('Strain_data.xlsx', sheetname='Strain info',index_col=0)

In [ ]:
META

In [ ]:
from sklearn.datasets import load_iris
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(base, )

In [ ]:
import graphviz 
exp= tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(exp) 
graph.render("output") 

In [ ]:
clf= tree.export_graphviz(clf, out_file=None, 
                         feature_names=data_feaures,  
                         class_names=data_target,  
                         filled=True, rounded=True,  
                         special_characters=True)  
graph2 = graphviz.Source(clf)  
graph2.render("Disease Decision Tree")
graph2

In [ ]:
iris